# Auto Encoders (Movie Recommender System)

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

*Importing dataset*

In [2]:
movies = pd.read_csv('dataset/ml-1m/movies.dat', sep='::', header=None, engine='python', encoding='latin-1')

In [3]:
users = pd.read_csv('dataset/ml-1m/users.dat', sep='::', header=None, engine='python', encoding='latin-1')

In [4]:
ratings = pd.read_csv('dataset/ml-1m/ratings.dat', sep='::', header=None, engine='python', encoding='latin-1')

*Creating the training and test set*

In [5]:
training_set = pd.read_csv('dataset/ml-100k/u1.base', delimiter='\t')

In [6]:
training_set = np.array(training_set, dtype='int')

In [7]:
test_set = np.array(pd.read_csv('dataset/ml-100k/u1.test', delimiter='\t'), dtype='int')

*Finding total number of movies and users*

In [8]:
total_users = int(max(max(training_set[:, 0], ), max(test_set[:, 0])))
total_movies = int(max(max(training_set[:, 1], ), max(test_set[:, 1])))

*Converting data into movies as columns and users as rows*

In [9]:
def convert(data):
  new_data = []
  for id_users in range(1, total_users + 1):
    id_movies = data[:, 1] [data[:, 0] == id_users]
    id_ratings = data[:, 2] [data[:, 0] == id_users]
    ratings = np.zeros(total_movies)
    ratings[id_movies - 1] = id_ratings
    new_data.append(list(ratings))
  return new_data
training_set = convert(training_set)
test_set = convert(test_set)

*Converting data into torch tensors*

In [10]:
training_set = torch.FloatTensor(training_set).cuda()
test_set = torch.FloatTensor(test_set).cuda()

*Creating Auto Encoder architecture and training it*

In [11]:
class StackedAutoEncoder (nn.Module):
    def __init__(self, visible_nodes, hidden_nodes, activation_obj, ):
        super(StackedAutoEncoder, self).__init__()
        self.full_connection_1 = nn.Linear(visible_nodes, hidden_nodes[0])
        self.full_connection_2 = nn.Linear(hidden_nodes[0], hidden_nodes[1])
        self.full_connection_3 = nn.Linear(hidden_nodes[1], hidden_nodes[2])
        self.full_connection_4 = nn.Linear(hidden_nodes[2], hidden_nodes[1])
        self.full_connection_5 = nn.Linear(hidden_nodes[1], hidden_nodes[0])
        self.full_connection_6 = nn.Linear(hidden_nodes[0], visible_nodes)
        self.activation = activation_obj

    def forward(self, x):
        x = self.activation(self.full_connection_1(x))
        x = self.activation(self.full_connection_2(x))
        x = self.activation(self.full_connection_3(x))
        x = self.activation(self.full_connection_4(x))
        x = self.activation(self.full_connection_5(x))
        x = self.full_connection_6(x)
        return x

In [12]:
visible_nodes = total_movies
hidden_nodes = [40, 20, 10]
activation = nn.Sigmoid()

In [13]:
sae = StackedAutoEncoder(visible_nodes, hidden_nodes, activation)
sae.cuda()

StackedAutoEncoder(
  (full_connection_1): Linear(in_features=1682, out_features=40, bias=True)
  (full_connection_2): Linear(in_features=40, out_features=20, bias=True)
  (full_connection_3): Linear(in_features=20, out_features=10, bias=True)
  (full_connection_4): Linear(in_features=10, out_features=20, bias=True)
  (full_connection_5): Linear(in_features=20, out_features=40, bias=True)
  (full_connection_6): Linear(in_features=40, out_features=1682, bias=True)
  (activation): Sigmoid()
)

In [14]:
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr=0.01, weight_decay=0.5)

In [15]:
epochs = 200
for epoch in range(epochs):
    training_loss = 0
    s = 0.
    for user in range(total_users):
        input = Variable(training_set[user]).unsqueeze(0)
        target = input.clone()
        if torch.sum(target.data > 0) > 0:
            output = sae(input)
            target.requires_grad = False
            output[target == 0] = 0
            loss = criterion(output, target)
            mean_corrector = total_movies / float(torch.sum(target.data > 0) + 1e-10)
            loss.backward()
            training_loss += np.sqrt(loss.data.item() * mean_corrector)
            s += 1.
            optimizer.step()
    print('epoch:', epoch + 1, 'loss:', training_loss / s, 'star(s) out of 5')

epoch: 1 loss: 1.5488275838458894 star(s) out of 5
epoch: 2 loss: 1.0750408583731579 star(s) out of 5
epoch: 3 loss: 1.0560077350071018 star(s) out of 5
epoch: 4 loss: 1.0495526581573273 star(s) out of 5
epoch: 5 loss: 1.0475762864192593 star(s) out of 5
epoch: 6 loss: 1.0442852473819553 star(s) out of 5
epoch: 7 loss: 1.044380209905098 star(s) out of 5
epoch: 8 loss: 1.0429923255580906 star(s) out of 5
epoch: 9 loss: 1.0430915748804348 star(s) out of 5
epoch: 10 loss: 1.0416671804259403 star(s) out of 5
epoch: 11 loss: 1.0415199138304836 star(s) out of 5
epoch: 12 loss: 1.0391496874563853 star(s) out of 5
epoch: 13 loss: 1.0387104801419318 star(s) out of 5
epoch: 14 loss: 1.0374491959206724 star(s) out of 5
epoch: 15 loss: 1.0375545505694435 star(s) out of 5
epoch: 16 loss: 1.035612345441371 star(s) out of 5
epoch: 17 loss: 1.0354491157899632 star(s) out of 5
epoch: 18 loss: 1.033688676401667 star(s) out of 5
epoch: 19 loss: 1.032870619036821 star(s) out of 5
epoch: 20 loss: 1.0332896

*Testing the Auto Encoder*

In [16]:

test_loss = 0
s = 0.
for user in range(total_users):
    input = Variable(training_set[user]).unsqueeze(0)
    target = Variable(test_set[user]).unsqueeze(0)
    if torch.sum(target.data > 0) > 0:
        output = sae(input)
        target.requires_grad = False
        output[target == 0] = 0
        loss = criterion(output, target)
        mean_corrector = total_movies / float(torch.sum(target.data > 0) + 1e-10)
        test_loss += np.sqrt(loss.data.item() * mean_corrector)
        s += 1.
print('Loss on Test Set:', test_loss / s, 'star(s) out of 5')

Loss on Test Set: 0.9638131935324882 star(s) out of 5
